**以头条新闻的分类数据集为语料来实现**

### 数据处理
#### 读取数据与分词

- 用两种工具处理：jieba和thulac,进行分词
- 然后去除停用词，用的百度停用词数据集

**Todo**：

    1.分词的效果不咋样啊，很多专有名词识别不出来
    2.类别不均衡，需要增强

本次只比较不同词向量的效果，暂时不做这些

In [1]:
import jieba 
import thulac
import re 
import pandas as pd 
import numpy as np 
import gc 

In [2]:
filename = 'toutiao_cat_data.txt'
thu1 = thulac.thulac(seg_only=True) 
stopwords =[ line.rstrip() for line in open('stopwords-master/baidu_stopwords.txt',encoding='utf-8') ]

def jieba_pre(text):
    seg_list = jieba.cut(text, cut_all=False,HMM=True)
    return [seg for seg in seg_list if seg not in stopwords]

def thu_pre(text):
     #默认模式是分词+词性标注，这里只分词
    seg_list = thu1.cut(text, text=True)  #进行一句话分词
    return [seg for seg in seg_list.split(' ')  if seg not in stopwords]

def preprocess(pre_model,filename):
    texts,labels =[],[]
    with open(filename, encoding='utf-8') as f:
        for l in f.readlines():
            s = l.strip().split('_!_')
            if len(s) >=4 :
                ss = re.sub('[0-9]+',' ',s[3])
                texts.append(pre_model(ss)+s[4].split(','))
                labels.append(s[1])
    return texts,labels 
            


Model loaded succeed


In [3]:
jieba_texts,jieba_labels = preprocess(jieba_pre,filename)
thu_texts,thu_labels = preprocess(thu_pre,filename)
gc.collect()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yui\AppData\Local\Temp\jieba.cache
Loading model cost 0.742 seconds.
Prefix dict has been built successfully.


0

In [2]:
#jieba_texts,jieba_labels = preprocess(jieba_pre,filename)
#thu_texts,thu_labels = preprocess(thu_pre,filename)
'''
好慢！以防万一导出到本地

jieba_seq=[]
for i in range(len(jieba_texts)):
    jieba_seq.append(','.join(jieba_texts[i])+','+jieba_labels[i])

thu_seq=[]
for i in range(len(jieba_texts)):
    thu_seq.append(','.join(thu_texts[i])+','+thu_labels[i])

    
import pandas as pd 
import numpy as np
df = pd.DataFrame(data=np.array([jieba_seq,thu_seq]).transpose(),columns=['jieba_seq','thu_seq'])
df.to_csv('fenci_pre.csv',encoding='utf-8')


df = pd.read_csv('fenci_pre.csv',encoding='utf-8')
jieba_texts,jieba_labels =[],[]
thu_texts,thu_labels=[],[]
for i in range(df.shape[0]):
    jieba_texts.append(df.iloc[i][1].split(',')[:-1])
    thu_texts.append(df.iloc[i][2].split(',')[:-1])
    jieba_labels.append(df.iloc[i][1].split(',')[-1])
    thu_labels.append(df.iloc[i][2].split(',')[-1])

'''

#### 词表字典

这一步得到三个产物：

    单词计数
    单词到索引
    索引到单词

In [4]:
def get_vocab(corpus):
    word_counts = {}
    word_to_index = {}
    index_to_word = {}


    for line in corpus:
        for word in line:
            if word in word_counts and len(word)>1  :
                word_counts[word]+=1
            elif len(word)>1 :
                word_counts[word] = 1 
    word_counts=dict(sorted(word_counts.items(),key=lambda x:x[1],reverse=True)[:2000])
    for  idx,item in enumerate(word_counts.items()):
        word_to_index[item[0]] = idx 
    
    index_to_word = dict([ (v,k) for k,v in word_to_index.items() ])

    return word_counts,word_to_index,index_to_word

In [5]:
jb_word_counts,jb_word_to_index,jb_index_to_word = get_vocab(jieba_texts)

In [6]:
#thu_word_counts,thu_word_to_index,thu_index_to_word = get_vocab(thu_texts)

In [7]:
'''
标签编码
'''
labels_set = list(set(jieba_labels))
labels_set.sort()
labels_dict = dict([(lab,idx) for idx,lab in enumerate(labels_set)])

In [8]:
jieba_labels = [ labels_dict[i] for i in jieba_labels]
#thu_labels = [ labels_dict[i] for i in thu_labels]

In [9]:
labels_cnt ={}
for l in jieba_labels:
    if l  in labels_cnt.keys():
        labels_cnt[l] +=1
    else:
        labels_cnt[l]=1
labels_cnt

{1: 28031,
 2: 39396,
 3: 37568,
 4: 27085,
 5: 17672,
 6: 35785,
 7: 27058,
 8: 41543,
 9: 24984,
 10: 21422,
 11: 26909,
 13: 19322,
 14: 29300,
 12: 340,
 0: 6273}

#### 词向量
##### 词袋
- 做法：
    - 将语料库中的所有词提取出来形成词汇表
    - 按照某种顺序排列这些词
    - 对于每个文档，统计每个词在文档中出现的次数
- 形成文档-词条矩阵：每个元素体现的是词频
- 文本相似度计算：
    - 计算两个行向量之间的点积
        - 缺陷：只捕捉相似部分，而其他地方不受影响
    - 余弦相似度：
        - 将两个向量之间的点积除以欧几里得范数

In [10]:
def words_bag(corpus,word_to_index):
    seqs = []
    vocab_size = 2000
    for line in  corpus:
        seq = [0]*vocab_size 
        for word in line:
            try:
                seq[word_to_index.get(word)]+=1 
            except TypeError:
                pass 
        seqs.append(seq)
    return seqs

In [11]:
jb_wordbag_seqs = words_bag(jieba_texts,jb_word_to_index)
gc.collect()

0

In [12]:
#thu_wordbag_seqs = words_bag(thu_texts,thu_word_to_index)  
#gc.collect()

##### TF-IDF
- 词袋的局限性：
    - 将每个词的重要性同等对待
    - 实际上，在一个语料库中，某些词出现的频率会较高，会使得结果产生偏倚。
- TF-IDF:
     - 统计语料库中，含有每个词的文档数量（文档频率）
     - 然后将词频除以该词条的文档频率
     - 能够突出在文档中具有唯一性的词
- 还存在几种变形：标准化或者平滑处理等等

In [26]:
idf =[1]*2000

In [27]:
'''
内存有限，只看jieba的
'''
for line in jieba_texts:
    wordset = set(line)
    for word in wordset :
        if jb_word_to_index.get(word):
            idf[jb_word_to_index.get(word)]+=1 
idf = np.array(idf).reshape(1,2000)  

In [36]:
gc.collect()
jb_wordbag_seqs = np.array(jb_wordbag_seqs )
gc.collect()
idf_seqs = jb_wordbag_seqs/idf
gc.collect()

11007


##### one-hot

In [58]:
seqs = []
for line in jieba_texts:
    seq = []
    for word in line :
        if jb_word_to_index.get(word):
            seq.append(jb_word_to_index.get(word))
    seqs.append(seq)

max_len = max(len(seq) for seq in seqs)

'''
padding
'''
padding_seqs=[]
for line in seqs:
    padding_seqs.append(line+[0]*(max_len-len(line)))

##### 词嵌入
- word2vec：词嵌入框架之一
    - 核心概念：
        - 一个模型能根据相邻词预测中心词，反之亦然；从而能捕捉上下文的意思。
    - 每个词的意义分布在向量中，十分稳健。
    - 词向量大小为超参数（sk）。
    - 词袋则不一样，大小会随着数量增加而增加
    - 一旦预训练了一组词向量，便可有效使用，无需转换
    - 可作为RNN网络的输入向量
    - 看我的cs224笔记
- Glove:
    - 词表征法的Glove,全局向量。
    - 利用共现统计方法，直接优化每个词的向量表示法。
    - 实现：
        - 对于语料中所有的词对i,j，计算词j在词i的上下文出现的概率p(j|i);（j在i上下文出现：窗口内）。
        - 对每个词初始化两个随机向量，一个代表上下文$\widetilde{w}$，一个代表中心词$w$。
        - 对于任何词对i,j，想让他的词向量的点积$\widetilde{w_{i}}*w_{j}$等于共现概率
        - 以此为目标，利用恰当的损失函数，通过迭代的方式优化这些词向量。
        - 结果为一组词向量，捕捉了各个词之间的相似性与差异性
     - 另一个角度：
         实际上是将共现概率矩阵分解为两个小矩阵
     
 


In [67]:
'''
直接掉包~
'''
import gensim, logging, os
import logging  

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,filename='test_01.log')  


In [70]:
corpus =[]
for line in jieba_texts:
    corpus.append(' '.join(line))

In [ ]:

sentences = word2vec.LineSentence(corpus) 

model = word2vec.Word2Vec(sentences, size=12,window=25,min_count=2,workers=5,sg=1,hs=1) 
